In [10]:
from flask import Flask, render_template, make_response, request, render_template_string,redirect,url_for,jsonify #, Markup
from markupsafe import Markup 
from flask_flatpages import FlatPages, pygmented_markdown
from flask_frozen import Freezer
from flask_minify import minify 
import markdown
# from yaml import safe_load
import yaml

import pandas as pd
# from datetime import date, timedelta, datetime
import subprocess
import numpy as np
import sys, os, re, shutil, glob
import copy
import polars as pl
import json

import geopy

from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
from typing import List, Dict, Tuple, Optional
import pandas as pd
from openpyxl import load_workbook
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import os
import math
import polars as pl
import polars.selectors as cs


In [11]:
def xlsx_sheet_to_df(filename:str,sheetname:str) -> pd.DataFrame:
    wb = load_workbook(filename)
    ws = wb[sheetname]
    data = list(ws.values)
    headers=next(iter(data))
    df_out = pd.DataFrame(data[1:],columns=headers)
    return(df_out)


In [14]:
df_months = xlsx_sheet_to_df(filename=r'../data/months.xlsx',sheetname='Months')
df_countries = xlsx_sheet_to_df(filename=r'../data/countries.xlsx',sheetname='Countries')

In [22]:
df_countries

,NL,EN
0,Egypte,Egypt
1,Griekenland,Greece
2,Indonesië,Indonesia
3,Italië,Italy
4,Nederland,Netherlands
5,Nederlandse Antillen,Dutch Antilles
6,Portugal,Portugal
7,Spanje,Spain
8,Turkije,Turkey


In [27]:
# def create_dict_from_df(df_translations,base='NL'):
#     cols = list(df_translations.columns)
#     dict_out={}
#     for i,v in df_translations.iterrows():
        

# create_dict_from_df(df_countries)

In [51]:
df_destinations = xlsx_sheet_to_df(filename=r'../data/destinations.xlsx',sheetname='Destinations')

In [52]:
def create_dict_destinations(df_destinations):
    cols = list(df_destinations.columns)
    dict_destinations = {}
    for i,v in df_destinations.iterrows():
        country = v['Country'].lower().replace(' ','-')
        place = v['Place'].lower().replace(' ','-')
        lat = v['Lat']
        if country not in dict_destinations:
            dict_destinations[country]={}
        dict_destinations[country][place] = {col:v[col] for col in cols}
        dict_destinations[country][place]['Country_NL']=v['Country']
        dict_destinations[country][place]['Place_NL']=v['Place']
        dict_destinations[country][place]['data_country']=country
        dict_destinations[country][place]['data_place']=place
        
        # if lat is None or np.isnan(lat):
        #     dict_latlon = add_latlon(country=v['Country'],place=v['Place'])
        #     dict_destinations[country][place].update(dict_latlon)
    return(dict_destinations)

In [53]:
dict_destinations = create_dict_destinations(df_destinations)

In [54]:
dict_destinations

{'egypte': {'alexandrië': {'Country': 'Egypte',
   'Place': 'Alexandrië',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypte',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'},
  'dahab': {'Country': 'Egypte',
   'Place': 'Dahab',
   'Region': None,
   'Type': 'zon',
   'Lat': 28.4963633,
   'Lon': 34.5145652,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopula

In [61]:
def translate_dict_destinations(dict_destinations,lang='NL'):
    dict_out = {}
    if lang != 'NL':
        for c in dict_destinations:
            for p in dict_destinations[c]:
                Ct = dict_destinations[c][p]['Country_'+lang]
                ct = Ct.replace(' ','-').lower()
                if ct not in dict_out:
                    dict_out[ct]={}
                Pt = dict_destinations[c][p]['Place_'+lang]
                pt = Pt.replace(' ','-').lower()
                dict_out[ct][pt]=dict_destinations[c][p]
                dict_out[ct][pt]['Country']=dict_out[ct][pt]['Country_'+lang]
                dict_out[ct][pt]['Place']=dict_out[ct][pt]['Place_'+lang]
                dict_out[ct][pt]['Region']=dict_out[ct][pt]['Region_'+lang]
    else:
        dict_out=dict_destinations
    return dict_out

dict_destinations_EN = translate_dict_destinations(dict_destinations,lang='EN')
dict_destinations_EN

{'egypt': {'alexandria': {'Country': 'Egypt',
   'Place': 'Alexandria',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypt',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'},
  'dahab': {'Country': 'Egypt',
   'Place': 'Dahab',
   'Region': None,
   'Type': 'zon',
   'Lat': 28.4963633,
   'Lon': 34.5145652,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 

In [239]:
def create_dict_destinations_data(dict_destinations):
    dict_out = {}
    for c in dict_destinations:
        for p in dict_destinations[c]:
            c_data = dict_destinations[c][p]['data_country']
            if c_data not in dict_out:
                dict_out[c_data]={}
            p_data = dict_destinations[c][p]['data_place']
            dict_out[c_data][p_data]=dict_destinations[c][p]
            
    return dict_out

create_dict_destinations_data(dict_destinations_EN)

{'egypte': {'alexandrië': {'Country': 'Egypt',
   'Place': 'Alexandria',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypt',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'},
  'dahab': {'Country': 'Egypt',
   'Place': 'Dahab',
   'Region': None,
   'Type': 'zon',
   'Lat': 28.4963633,
   'Lon': 34.5145652,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular':

In [76]:

def clean_dict_destinations(dict_destinations,folder_data_processed='./data/processed'):
    dict_destinations_clean = {}
    for country in dict_destinations:
        for place in dict_destinations[country]:
            boolean_add_key = True
            c = dict_destinations[country][place]['data_country']
            p = dict_destinations[country][place]['data_place']
            # print(f'{c}_{p}')
            # print(boolean_add_key)
            fp = os.path.join(folder_data_processed,c,p)
            if os.path.exists(fp):
                # print(f'YES: {c}_{p}')
                list_csvs = ['df_mm3d.csv','df_mm3h.csv']
                for lc in list_csvs:
                    fpc=os.path.join(fp,lc)
                    # print(fpc)
                    if os.path.exists(fpc):
                        
                        if os.path.getsize(fpc) <= 5:
                            boolean_add_key = False   
                    else:
                        boolean_add_key = False
                # print(boolean_add_key)
            else:
                # print(f'NO: {c}_{p}')
                boolean_add_key = False
            # print(boolean_add_key)
            if boolean_add_key:
                if (country in dict_destinations_clean):
                    dict_destinations_clean[country].update({
                        place:dict_destinations[country][place]
                        })
                else:
                    dict_destinations_clean.update({
                        country:{place:dict_destinations[country][place]}
                        })
    return dict_destinations_clean


d2 = clean_dict_destinations(dict_destinations_EN,folder_data_processed='../data/processed')
# d2
for c in d2:
    for p in d2[c]:
        print(p)

alexandria
cairo
el-gouna
hurghada
luxor
makadi-bay
sharm-el-sheikh
athens
chersonissos
chios
corfu
faliraki
heraklion
kalamata
karpathos
kefalonia
kokkari
kos
lefkas
limnos
malia
mykonos
naxos
nidri
rethymnon
rhodes
saloniki
samos
santorini
sitia
skiathos
stalis
thassos
thessaloniki
zakynthos
balikpapan
bandung
denpasar
flores
jakarta
lombok
makassar
medan
sanur
ubud
yogyakarta
bardolino
bari
bologna
brindisi
cannobio
caorle
cinque-terre
lake-como
lake-garda
genua
lake-maggiore
livigno
merano
milan
naples
ortisei
peschiera
piedmont
pisa
porlezza
apulia
rimini
rome
sardinia
sicily
sorrento
tuscany
turin
venice
verona
ede
bonaire
curacao
albufeira
azores
braga
carvoeiro
cascais
castelo-branco
coimbra
ericeira
estoril
faro
figueira-da-foz
funchal
lagos
lisbon
madeira
nazare
ponta-delgada
porto
tavira
alicante
almeria
barcelona
benalmadena
bilbao
blanes
cadiz
calella
cambrils
cordoba
finestrat
fuengirola
fuerteventura
girona
gran-canaria
guardamar
ibiza
lanzarote
lloret-de-mar
madrid
mala

In [77]:
def create_dict_countries(dict_destinations_clean):
   dict_countries={}
   for k1,v1 in sorted(dict_destinations_clean.items()):
      
      k2s = sorted(v1.keys())
      dict_countries[k1]={
         'country_name':dict_destinations_clean[k1][k2s[0]]['Country'],
         'n_places':len(list(k2s)),
         'list_place_slugs':list(k2s),
         'list_place_names':[dict_destinations_clean[k1][k2]['Place'] for k2 in k2s]
         }
   return dict_countries

In [78]:
create_dict_countries(d2)

{'egypt': {'country_name': 'Egypt',
  'n_places': 7,
  'list_place_slugs': ['alexandria',
   'cairo',
   'el-gouna',
   'hurghada',
   'luxor',
   'makadi-bay',
   'sharm-el-sheikh'],
  'list_place_names': ['Alexandria',
   'Cairo',
   'El Gouna',
   'Hurghada',
   'Luxor',
   'Makadi Bay',
   'Sharm el Sheikh']},
 'greece': {'country_name': 'Greece',
  'n_places': 28,
  'list_place_slugs': ['athens',
   'chersonissos',
   'chios',
   'corfu',
   'faliraki',
   'heraklion',
   'kalamata',
   'karpathos',
   'kefalonia',
   'kokkari',
   'kos',
   'lefkas',
   'limnos',
   'malia',
   'mykonos',
   'naxos',
   'nidri',
   'rethymnon',
   'rhodes',
   'saloniki',
   'samos',
   'santorini',
   'sitia',
   'skiathos',
   'stalis',
   'thassos',
   'thessaloniki',
   'zakynthos'],
  'list_place_names': ['Athens',
   'Chersonissos',
   'Chios',
   'Corfu',
   'Faliraki',
   'Heraklion',
   'Kalamata',
   'Karpathos',
   'Kefalonia',
   'Kokkari',
   'Kos',
   'Lefkas',
   'Limnos',
   'Mali

In [83]:
dict_destinations_EN

{'egypt': {'alexandria': {'Country': 'Egypt',
   'Place': 'Alexandria',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypt',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'},
  'dahab': {'Country': 'Egypt',
   'Place': 'Dahab',
   'Region': None,
   'Type': 'zon',
   'Lat': 28.4963633,
   'Lon': 34.5145652,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 

In [90]:
country='spain'
place1 = list(dict_destinations_EN[country].keys())[0]
data_country = dict_destinations_EN[country][place1]['data_country']
# dict_destinations_EN[country][place1]['data_country']


'spanje'

In [241]:
country='spain'
df_name = 'df_m_country'
folder_data_processed = '../data/processed'

def get_data_country(df_name,country,dict_destinations):
    place_dummy = list(dict_destinations[country].keys())[0]
    country_data = dict_destinations[country][place_dummy]['data_country']
    df_data_country = pd.read_csv(os.path.join(folder_data_processed,country_data,df_name+'.csv'))
    return(df_data_country)

df_m_country = get_data_country('df_m_country',country,dict_destinations_EN)
df_m_country

,m,tmin_med,tavg_med,tmax_med,wspd_med,rain_sum,rain_pct,prcp_sum,country,place
0,1,7.5,11.3,17.2,15.1,4.0,0.14,4.0,spanje,alicante
1,2,7.9,12.5,18.5,14.4,4.0,0.12,2.0,spanje,alicante
2,3,9.9,14.4,19.5,15.7,9.0,0.29,38.0,spanje,alicante
3,4,12.1,16.6,21.3,13.6,8.0,0.27,31.0,spanje,alicante
4,5,15.2,20.0,24.8,12.5,5.0,0.17,23.0,spanje,alicante
...,...,...,...,...,...,...,...,...,...,...
427,8,22.9,27.3,32.5,11.1,2.0,0.08,1.0,spanje,villajoyosa
428,9,20.2,24.1,29.5,11.5,7.0,0.25,24.0,spanje,villajoyosa
429,10,15.5,20.4,25.6,10.8,5.0,0.16,28.0,spanje,villajoyosa
430,11,12.0,15.8,22.0,13.7,6.0,0.18,12.0,spanje,villajoyosa


In [243]:
import polars as pl

# df_m_country = pl.from_pandas(df_m_country)
# df_m_country

In [244]:
df_m_country.group_by(['place']).agg(
    pl.col(['tavg_med']).min().alias('avg_min'),
    pl.col(['tmax_med']).min().alias('max_min'),
    pl.col(['tmax_med']).max().alias('max_max'),
    pl.col(['tmax_med']).mean().round(1).alias('max_avg'),
).sort(['max_avg'],descending=[True])

t=20
t_delta=2
c_dry=10
df_around = df_m_country.filter((pl.col('tavg_med')>=(t)) & (pl.col('tavg_med')<=(t+t_delta)) & (pl.col('prcp_sum')<c_dry)).sort(
    ['m','prcp_sum','tavg_med'],descending=[False,False,True])
df_m_around = df_around.group_by(['m']).agg(pl.col('place').len().alias('n_places')).filter(pl.col('n_places')>=3).sort(['n_places'],descending=True)

df_around = df_around.join(df_m_around,on='m',how='inner')

df_p_hot = df_m_country.group_by(['place']).agg(pl.col('tmax_med').top_k(3).mean().round(1).alias('tmax_top3'))
df_p_hot3 = df_m_country.select(pl.all().top_k_by('tmax_med',k=3).over('place', mapping_strategy="explode"))
df_p_hot = df_p_hot3.group_by(['place']).agg(pl.col('m').alias('list_m'),pl.col('tmax_med').alias('list_max')).join(df_p_hot,how='inner',on='place').sort(['tmax_top3'],descending=[True])
df_p_hot

AttributeError: 'DataFrame' object has no attribute 'group_by'

In [235]:
top_n = 3
[list(x) for x in df_p_hot['list_max'][:top_n]]

[[38.3, 38.2, 33.1], [36.0, 35.6, 32.1], [36.0, 35.6, 32.1]]

In [252]:
dict_c = {
    'top_n':3,
    'months_n_always':10,
    'degrees_thresholds':[20,25,30],
    'degrees_delta':2,
    'c_dry':10,
    'c_warm':18,'c_hot':38,
}

def create_dictQA_country(df_m_country,**kwargs):
    df_m_country = pl.from_pandas(df_m_country)
    dict_out={}
    top_n = kwargs['top_n']

    df_m_temps = df_m_country.group_by(['m']).agg(
        pl.col('tavg_med').mean().round(1).alias('mean'),
        pl.col('tavg_med').min().alias('min'),
        pl.col('tavg_med').max().alias('max')).sort(
        ['mean','max'],descending=[True,True])
    dict_out['month_warmest'] = {'m':list(df_m_temps['m'][:top_n]),
                                 'mean':list(df_m_temps['mean'][:top_n]),
                                 'min':list(df_m_temps['min'][:top_n]),
                                 'max':list(df_m_temps['max'][:top_n])}
    dict_out['month_coldest'] = {'m':list(df_m_temps['m'][::-1][:top_n]),
                                 'mean':list(df_m_temps['mean'][::-1][:top_n]),
                                 'min':list(df_m_temps['min'][::-1][:top_n]),
                                 'max':list(df_m_temps['max'][::-1][:top_n])}

    df_p_hot = df_m_country.group_by(['place']).agg(pl.col('tmax_med').top_k(3).mean().round(1).alias('tmax_top3'))
    df_p_hot3 = df_m_country.select(pl.all().top_k_by('tmax_med',k=3).over('place', mapping_strategy="explode"))
    df_p_hot = df_p_hot3.group_by(['place']).agg(pl.col('m').alias('list_m'),pl.col('tmax_med').alias('list_max')).join(df_p_hot,how='inner',on='place').sort(['tmax_top3'],descending=[True])
    dict_out['where_warmest'] = {'place':list(df_p_hot['place'][:top_n]),
                                 'max':[list(x) for x in df_p_hot['list_max'][:top_n]],
                                 'm':[list(x) for x in df_p_hot['list_m'][:top_n]]}
    df_p_cold = df_m_country.group_by(['place']).agg(pl.col('tmax_med').bottom_k(k=3).mean().round(1).alias('tmax_bottom3'))
    df_p_cold3 = df_m_country.select(pl.all().bottom_k_by('tmax_med',k=3).over('place', mapping_strategy="explode"))
    # print(df_p_cold3)
    df_p_cold = df_p_cold3.group_by(['place']).agg(pl.col('m').alias('list_m'),pl.col('tmax_med').alias('list_max')).join(df_p_cold,how='inner',on='place').sort(['tmax_bottom3'],descending=[False])
    print(df_p_cold)
    dict_out['where_coldest'] = {'place':list(df_p_cold['place'][:top_n]),
                                 'max':[list(x) for x in df_p_cold['list_max'][:top_n]],
                                 'm':[list(x) for x in df_p_cold['list_m'][:top_n]]}

    for t in kwargs['degrees_thresholds']:
        df_m_always = df_m_country.filter(
            pl.col('tmax_med')>=t).group_by(['place']).agg(
            pl.col('place').len().alias('n')
        ).sort(['n'],descending=[True]).filter(pl.col('n')>=kwargs['months_n_always'])
        dict_out[f'where_max_always_{t}'] = {'place':list(df_m_always['place']),'n_months':list(df_m_always['n'])}

        df_around = df_m_country.filter(
            (pl.col('tavg_med')>=(t)) & (pl.col('tavg_med')<=(t+kwargs['degrees_delta'])) & (pl.col('prcp_sum')<kwargs['c_dry'])
        ).sort(
            ['m','prcp_sum','tavg_med'],descending=[False,False,True])
        df_m_around = df_around.group_by(['m']).agg(
            pl.col('place').len().alias('n_places')).filter(
            pl.col('n_places')>=top_n).sort(
            ['n_places'],descending=True)
        df_around = df_around.join(df_m_around,on='m',how='inner')
        dict_out[f'when_med_around_{t}'] = {'m':[m for m in df_m_around['m']],
                                            'place':[list(df_around.filter(pl.col('m')==m)['place']) for m in df_m_around['m']],
                                             'n_places':[df_m_around.filter(pl.col('m')==m)['n_places'][0] for m in df_m_around['m']]}
        

    
    return dict_out

create_dictQA_country(df_m_country,**dict_c)

shape: (36, 4)
┌───────────────────┬────────────┬────────────────────┬──────────────┐
│ place             ┆ list_m     ┆ list_max           ┆ tmax_bottom3 │
│ ---               ┆ ---        ┆ ---                ┆ ---          │
│ str               ┆ list[i64]  ┆ list[f64]          ┆ f64          │
╞═══════════════════╪════════════╪════════════════════╪══════════════╡
│ madrid            ┆ [1, 12, 2] ┆ [10.2, 11.5, 13.7] ┆ 11.8         │
│ bilbao            ┆ [1, 12, 2] ┆ [14.0, 15.4, 15.7] ┆ 15.0         │
│ barcelona         ┆ [1, 12, 2] ┆ [14.0, 15.0, 16.0] ┆ 15.0         │
│ sitges            ┆ [1, 2, 12] ┆ [14.4, 15.8, 16.0] ┆ 15.4         │
│ lloret-de-mar     ┆ [1, 12, 2] ┆ [14.7, 15.8, 16.5] ┆ 15.7         │
│ …                 ┆ …          ┆ …                  ┆ …            │
│ playa-de-santiago ┆ [1, 2, 3]  ┆ [21.0, 21.5, 21.5] ┆ 21.3         │
│ maspalomas        ┆ [1, 2, 3]  ┆ [21.3, 21.5, 22.1] ┆ 21.6         │
│ gran-canaria      ┆ [1, 2, 3]  ┆ [21.3, 21.5, 22.1] ┆ 21.6  

{'month_warmest': {'m': [8, 7, 9],
  'mean': [26.3, 26.0, 23.5],
  'min': [21.6, 21.4, 20.3],
  'max': [29.4, 29.2, 25.1]},
 'month_coldest': {'m': [1, 2, 12],
  'mean': [11.2, 12.7, 12.9],
  'min': [5.9, 8.6, 7.4],
  'max': [18.7, 19.0, 19.8]},
 'where_warmest': {'place': ['cordoba', 'murcia', 'orihuela'],
  'max': [[38.3, 38.2, 33.1], [36.0, 35.6, 32.1], [36.0, 35.6, 32.1]],
  'm': [[8, 7, 6], [8, 7, 6], [8, 7, 6]]},
 'where_coldest': {'place': ['madrid', 'bilbao', 'barcelona'],
  'max': [[10.2, 11.5, 13.7], [14.0, 15.4, 15.7], [14.0, 15.0, 16.0]],
  'm': [[1, 12, 2], [1, 12, 2], [1, 12, 2]]},
 'where_max_always_20': {'place': ['playa-de-santiago',
   'lanzarote',
   'fuerteventura',
   'maspalomas',
   'gran-canaria',
   'orihuela',
   'murcia'],
  'n_months': [12, 12, 12, 12, 12, 10, 10]},
 'when_med_around_20': {'m': [5, 11, 4],
  'place': [['fuerteventura',
    'gran-canaria',
    'maspalomas',
    'lanzarote',
    'playa-de-santiago',
    'almeria',
    'sevilla',
    'cordoba',

In [198]:
dict_destinations


{'egypte': {'alexandrië': {'Country': 'Egypt',
   'Place': 'Alexandria',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypt',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'},
  'dahab': {'Country': 'Egypt',
   'Place': 'Dahab',
   'Region': None,
   'Type': 'zon',
   'Lat': 28.4963633,
   'Lon': 34.5145652,
   'Map': 1,
   'Label': 'b',
   'Popular': 0,
   'Superpopular':

In [270]:
from openpyxl import load_workbook

def xlsx_sheet_to_df(filename:str,sheetname:str) -> pd.DataFrame:
    wb = load_workbook(filename)
    ws = wb[sheetname]
    data = list(ws.values)
    headers=next(iter(data))
    df_out = pd.DataFrame(data[1:],columns=headers)
    return(df_out)


folder_data = '../data'
fn = os.path.join(folder_data,'sentences.xlsx')

df_sentences_c = xlsx_sheet_to_df(fn,'country')
df_sentences_c

,ID,NL,EN
0,h2_rain,Regen in {country_name},Rain in {country_name}
1,tile_month_warmest,Warmste maanden,Warmest months
2,tile_month_coldest,Koudste maanden,Coldest months
3,tile_where_warmest,Warmste plekken,Warmest places
4,tile_where_coldest,Koudste plekken,Coldest places
5,faq_month_warmest_h3,Wat zijn de warmste maanden in {country_name}?,What are the warmest months in {country_name}?
6,faq_month_warmest_span1,De warmste maanden zijn,The warmest months are
7,faq_month_coldest_h3,Wat zijn de koudste maanden in {country_name}?,What are the coldest months in {country_name}?
8,faq_month_coldest_span1,De koudste maanden zijn,The coldest months are
9,faq_where_max_always_20_h3,Waar in {country_name} is het het hele jaar do...,Where in {country_name} is it at least 20 degr...


In [271]:
lang = 'NL'
dict_vars_c = {'country_name':'Egypte','n_places':str(3)}

df_sentences_c[['ID',lang]]

,ID,NL
0,h2_rain,Regen in {country_name}
1,tile_month_warmest,Warmste maanden
2,tile_month_coldest,Koudste maanden
3,tile_where_warmest,Warmste plekken
4,tile_where_coldest,Koudste plekken
5,faq_month_warmest_h3,Wat zijn de warmste maanden in {country_name}?
6,faq_month_warmest_span1,De warmste maanden zijn
7,faq_month_coldest_h3,Wat zijn de koudste maanden in {country_name}?
8,faq_month_coldest_span1,De koudste maanden zijn
9,faq_where_max_always_20_h3,Waar in {country_name} is het het hele jaar do...


In [273]:
import re

def create_dict_ID(df_sentences,dict_vars,lang='NL'):
    dict_out = {}
    for i,r in df_sentences.iterrows():
        id = r['ID']
        s = r[lang]
        for k in dict_vars:
            pattern = '{'+k+'}'
            s = re.sub(pattern,dict_vars[k],s)
        dict_out[id] = s
    return dict_out

dict_ID = create_dict_ID(df_sentences_c,dict_vars_c,lang='NL')
dict_ID

{'h2_rain': 'Regen in Egypte',
 'tile_month_warmest': 'Warmste maanden',
 'tile_month_coldest': 'Koudste maanden',
 'tile_where_warmest': 'Warmste plekken',
 'tile_where_coldest': 'Koudste plekken',
 'faq_month_warmest_h3': 'Wat zijn de warmste maanden in Egypte?',
 'faq_month_warmest_span1': 'De warmste maanden zijn',
 'faq_month_coldest_h3': 'Wat zijn de koudste maanden in Egypte?',
 'faq_month_coldest_span1': 'De koudste maanden zijn',
 'faq_where_max_always_20_h3': 'Waar in Egypte is het het hele jaar door minimaal 20 graden?',
 'faq_where_max_always_20_span1': 'Plaatsen waar de dagelijkse maximum temperatuur vrijwel het hele jaar minimaal 20 graden is',
 'faq_where_max_always_25_h3': 'Waar in Egypte is het het hele jaar door minimaal 25 graden?',
 'faq_where_max_always_25_span1': 'Plaatsen waar het vrijwel het hele jaar minimaal 25 graden is',
 'faq_where_max_always_30_h3': 'Waar in Egypte is het het hele jaar door minimaal 30 graden?',
 'faq_where_max_always_30_span1': 'Plaatsen 

In [277]:
# TO DO:
def create_dict_anchors(list_ids,dict_id):
    dict_out = {}
    for id in list_ids:
        dict_out[id] = {
            'el':id.split('_')[0],
            'text':dict_id[id]
        }
    return dict_out

list_ids = ['h2_temp','h3_temp_avg','h4_temp_avg_map','h4_temp_avg_table','h3_temp_max','h4_temp_max_map','h4_temp_max_table','h2_rain','h3_rain','h2_faq']
create_dict_anchors(list_ids,dict_ID)

{'h2_temp': {'el': 'h2', 'text': 'Temperatuur in Egypte'},
 'h3_temp_avg': {'el': 'h3', 'text': 'Gemiddelde temperatuur per maand'},
 'h4_temp_avg_map': {'el': 'h4',
  'text': 'Kaart (animatie) met gemiddelde temperatuur in Egypte'},
 'h4_temp_avg_table': {'el': 'h4', 'text': 'Tabel met gemiddelde temperatuur'},
 'h3_temp_max': {'el': 'h3', 'text': 'Maximum temperatuur per maand'},
 'h4_temp_max_map': {'el': 'h4',
  'text': 'Kaart (animatie) met maximum temperatuur in Egypte'},
 'h4_temp_max_table': {'el': 'h4', 'text': 'Tabel met maximum temperatuur'},
 'h2_rain': {'el': 'h2', 'text': 'Regen in Egypte'},
 'h3_rain': {'el': 'h3', 'text': 'Regen per maand'},
 'h2_faq': {'el': 'h2', 'text': 'Veelgestelde vragen over het weer in Egypte'}}

In [282]:
folder_data_processed

'../data/processed'

In [285]:
list_months3 = ['jan','feb','mrt','apr','mei','jun','jul','aug','sep','okt','nov','dec']

In [286]:
def get_data_place(df_name,country,place):
      df_data_place = pd.read_csv(os.path.join(folder_data_processed,country,place,df_name+'.csv'))
      df_data_place['m_abbr']=df_data_place['m'].map(lambda x: list_months3[x-1])
      return df_data_place

In [288]:
df_mm3 = get_data_place('df_mm3',country='spanje',place='madrid')
df_m = get_data_place('df_m',country='spanje',place='madrid')

In [289]:
df_mm3.head()

,m,m3,tmin_med,tavg_med,tmax_med,wspd_med,rain_sum,rain_pct,prcp_sum,m_abbr
0,1,1,2.6,5.7,10.0,9.7,2.0,0.23,10.0,jan
1,1,2,1.6,4.7,9.8,10.6,3.0,0.26,6.0,jan
2,1,3,5.4,8.4,12.4,10.7,3.0,0.32,6.0,jan
3,2,1,4.1,7.5,12.2,11.7,2.0,0.25,11.0,feb
4,2,2,5.4,9.2,14.5,10.2,1.0,0.09,0.0,feb


In [290]:
df_m.head()

,m,tmin_med,tavg_med,tmax_med,wspd_med,rain_sum,rain_pct,prcp_sum,m_abbr
0,1,2.7,5.9,10.2,10.2,8.0,0.27,22.0,jan
1,2,5.0,8.8,13.7,11.1,5.0,0.18,12.0,feb
2,3,6.8,10.8,16.2,12.4,11.0,0.34,51.0,mrt
3,4,8.9,13.7,20.3,10.8,12.0,0.39,53.0,apr
4,5,12.1,17.9,24.8,11.6,7.0,0.23,17.0,mei


In [ ]:
dict_cp = {
    'top_n':3,
    'months_n_always':10,
    'degrees_thresholds':[20,25,30],
    'degrees_delta':2,
    'c_dry':10,
    'c_warm':18,'c_hot':38,
}

def create_dictQA_country_place(df_m_country,**kwargs):
    df_m_country = pl.from_pandas(df_m_country)
    dict_out={}
    top_n = kwargs['top_n']

    df_m_temps = df_m_country.group_by(['m']).agg(
        pl.col('tavg_med').mean().round(1).alias('mean'),
        pl.col('tavg_med').min().alias('min'),
        pl.col('tavg_med').max().alias('max')).sort(
        ['mean','max'],descending=[True,True])
    dict_out['month_warmest'] = {'m':list(df_m_temps['m'][:top_n]),
                                 'mean':list(df_m_temps['mean'][:top_n]),
                                 'min':list(df_m_temps['min'][:top_n]),
                                 'max':list(df_m_temps['max'][:top_n])}
    dict_out['month_coldest'] = {'m':list(df_m_temps['m'][::-1][:top_n]),
                                 'mean':list(df_m_temps['mean'][::-1][:top_n]),
                                 'min':list(df_m_temps['min'][::-1][:top_n]),
                                 'max':list(df_m_temps['max'][::-1][:top_n])}

    df_p_hot = df_m_country.group_by(['place']).agg(pl.col('tmax_med').top_k(3).mean().round(1).alias('tmax_top3'))
    df_p_hot3 = df_m_country.select(pl.all().top_k_by('tmax_med',k=3).over('place', mapping_strategy="explode"))
    df_p_hot = df_p_hot3.group_by(['place']).agg(pl.col('m').alias('list_m'),pl.col('tmax_med').alias('list_max')).join(df_p_hot,how='inner',on='place').sort(['tmax_top3'],descending=[True])
    dict_out['where_warmest'] = {'place':list(df_p_hot['place'][:top_n]),
                                 'max':[list(x) for x in df_p_hot['list_max'][:top_n]],
                                 'm':[list(x) for x in df_p_hot['list_m'][:top_n]]}
    df_p_cold = df_m_country.group_by(['place']).agg(pl.col('tmax_med').bottom_k(k=3).mean().round(1).alias('tmax_bottom3'))
    df_p_cold3 = df_m_country.select(pl.all().bottom_k_by('tmax_med',k=3).over('place', mapping_strategy="explode"))
    # print(df_p_cold3)
    df_p_cold = df_p_cold3.group_by(['place']).agg(pl.col('m').alias('list_m'),pl.col('tmax_med').alias('list_max')).join(df_p_cold,how='inner',on='place').sort(['tmax_bottom3'],descending=[False])
    print(df_p_cold)
    dict_out['where_coldest'] = {'place':list(df_p_cold['place'][:top_n]),
                                 'max':[list(x) for x in df_p_cold['list_max'][:top_n]],
                                 'm':[list(x) for x in df_p_cold['list_m'][:top_n]]}

    for t in kwargs['degrees_thresholds']:
        df_m_always = df_m_country.filter(
            pl.col('tmax_med')>=t).group_by(['place']).agg(
            pl.col('place').len().alias('n')
        ).sort(['n'],descending=[True]).filter(pl.col('n')>=kwargs['months_n_always'])
        dict_out[f'where_max_always_{t}'] = {'place':list(df_m_always['place']),'n_months':list(df_m_always['n'])}

        df_around = df_m_country.filter(
            (pl.col('tavg_med')>=(t)) & (pl.col('tavg_med')<=(t+kwargs['degrees_delta'])) & (pl.col('prcp_sum')<kwargs['c_dry'])
        ).sort(
            ['m','prcp_sum','tavg_med'],descending=[False,False,True])
        df_m_around = df_around.group_by(['m']).agg(
            pl.col('place').len().alias('n_places')).filter(
            pl.col('n_places')>=top_n).sort(
            ['n_places'],descending=True)
        df_around = df_around.join(df_m_around,on='m',how='inner')
        dict_out[f'when_med_around_{t}'] = {'m':[m for m in df_m_around['m']],
                                            'place':[list(df_around.filter(pl.col('m')==m)['place']) for m in df_m_around['m']],
                                             'n_places':[df_m_around.filter(pl.col('m')==m)['n_places'][0] for m in df_m_around['m']]}
        

    
    return dict_out

create_dictQA_country(df_m_country,**dict_c)

In [293]:
dict_in =  {'agistri': {'Country': 'Griekenland', 'Place': 'Agistri', 'Region': None, 'Type': 'zon', 'Lat': 37.6937039, 'Lon': 23.35005226086773, 'Map': 1, 'Label': 'b', 'Popular': 0, 'Superpopular': 0, 'Country_EN': 'Greece', 'Place_EN': 'Agistri', 'Region_EN': None, 'Continent': 'Europa', 'Continent_EN': 'Europe', 'Country_NL': 'Griekenland', 'Place_NL': 'Agistri', 'data_country': 'griekenland', 'data_place': 'agistri'}, 'athene': {'Country': 'Griekenland', 'Place': 'Athene', 'Region': None, 'Type': 'stad', 'Lat': 37.9755648, 'Lon': 23.7348324, 'Map': 1, 'Label': 'b', 'Popular': 0, 'Superpopular': 0, 'Country_EN': 'Greece', 'Place_EN': 'Athens', 'Region_EN': None, 'Continent': 'Europa', 'Continent_EN': 'Europe', 'Country_NL': 'Griekenland', 'Place_NL': 'Athene', 'data_country': 'griekenland', 'data_place': 'athene'}, 'chalkidiki': {'Country': 'Griekenland', 'Place': 'Chalkidiki', 'Region': None, 'Type': 'zon', 'Lat': 40.33096275, 'Lon': 23.36214621474508, 'Map': 1, 'Label': 'b', 'Popular': 0, 'Superpopular': 0, 'Country_EN': 'Greece', 'Place_EN': 'Chalkidiki', 'Region_EN': None, 'Continent': 'Europa', 'Continent_EN': 'Europe', 'Country_NL': 'Griekenland', 'Place_NL': 'Chalkidiki', 'data_country': 'griekenland', 'data_place': 'chalkidiki'}, 'chersonissos': {'Country': 'Griekenland', 'Place': 'Chersonissos', 'Region': None, 'Type': 'zon', 'Lat': 35.308168, 'Lon': 25.3720231, 'Map': 1, 'Label': 't', 'Popular': 1, 'Superpopular': 1, 'Country_EN': 'Greece', 'Place_EN': 'Chersonissos', 'Region_EN': None, 'Continent': 'Europa', 'Continent_EN': 'Europe', 'Country_NL': 'Griekenland', 'Place_NL': 'Chersonissos', 'data_country': 'griekenland', 'data_place': 'chersonissos'}}
keys_keep=['Place','Map','Label','Lat','Lon']

dict_out = {}

for place in dict_in:
    for k in keys_keep:
        dict_out[place]={k:dict_in[place][k] for k in keys_keep}

dict_out

{'agistri': {'Place': 'Agistri',
  'Map': 1,
  'Label': 'b',
  'Lat': 37.6937039,
  'Lon': 23.35005226086773},
 'athene': {'Place': 'Athene',
  'Map': 1,
  'Label': 'b',
  'Lat': 37.9755648,
  'Lon': 23.7348324},
 'chalkidiki': {'Place': 'Chalkidiki',
  'Map': 1,
  'Label': 'b',
  'Lat': 40.33096275,
  'Lon': 23.36214621474508},
 'chersonissos': {'Place': 'Chersonissos',
  'Map': 1,
  'Label': 't',
  'Lat': 35.308168,
  'Lon': 25.3720231}}

In [296]:
pd.DataFrame.from_dict(dict_out)

,agistri,athene,chalkidiki,chersonissos
Place,Agistri,Athene,Chalkidiki,Chersonissos
Map,1,1,1,1
Label,b,b,b,t
Lat,37.693704,37.975565,40.330963,35.308168
Lon,23.350052,23.734832,23.362146,25.372023


In [300]:
dict_out['agistri']

{'Place': 'Agistri',
 'Map': 1,
 'Label': 'b',
 'Lat': 37.6937039,
 'Lon': 23.35005226086773}

In [308]:
l=[{'place': 'alanya', 'm': 1, 'tmax_med': 15.6}, {'place': 'bodrum', 'm': 2, 'tmax_med': 16.5}, {'place': 'alanya', 'm': 3, 'tmax_med': 18.3}, {'place': 'alanya', 'm': 4, 'tmax_med': 22.7}, {'place': 'alanya', 'm': 5, 'tmax_med': 27.4}, {'place': 'alanya', 'm': 6, 'tmax_med': 30.4}, {'place': 'alanya', 'm': 7, 'tmax_med': 35.6}, {'place': 'alanya', 'm': 8, 'tmax_med': 34.5}, {'place': 'alanya', 'm': 9, 'tmax_med': 32.1}]
ddc = {'bodrum':{'X1':2,'Y1':5},'alanya':{'X1':-5,'Y1':8}}

[{**item,**ddc[item['place']]} for item in l]

# for i in l:
#     ddc_i = ddc[i['place']]
#     for j in ddc_i:
#         i[j]=ddc_i[j]

# l

[{'place': 'alanya', 'm': 1, 'tmax_med': 15.6, 'X1': -5, 'Y1': 8},
 {'place': 'bodrum', 'm': 2, 'tmax_med': 16.5, 'X1': 2, 'Y1': 5},
 {'place': 'alanya', 'm': 3, 'tmax_med': 18.3, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 4, 'tmax_med': 22.7, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 5, 'tmax_med': 27.4, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 6, 'tmax_med': 30.4, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 7, 'tmax_med': 35.6, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 8, 'tmax_med': 34.5, 'X1': -5, 'Y1': 8},
 {'place': 'alanya', 'm': 9, 'tmax_med': 32.1, 'X1': -5, 'Y1': 8}]

In [320]:
ddc = {'egypte': {'alexandrië': {'Country': 'Egypte', 'Place': 'Alexandrië', 'Region': None, 'Type': 'stad', 'Lat': 31.1991806, 'Lon': 29.8951716, 'Map': 1, 'Label': 'b', 'Popular': 1, 'Superpopular': 0, 'Country_EN': 'Egypt', 'Place_EN': 'Alexandria', 'Region_EN': None, 'Continent': 'Afrika', 'Continent_EN': 'Africa', 'Country_NL': 'Egypte', 'Place_NL': 'Alexandrië', 'data_country': 'egypte', 'data_place': 'alexandrië'}, 'cairo': {'Country': 'Egypte', 'Place': 'Cairo', 'Region': None, 'Type': 'stad', 'Lat': 30.0443879, 'Lon': 31.2357257, 'Map': 1, 'Label': 'b', 'Popular': 1, 'Superpopular': 0, 'Country_EN': 'Egypt', 'Place_EN': 'Cairo', 'Region_EN': None, 'Continent': 'Afrika', 'Continent_EN': 'Africa', 'Country_NL': 'Egypte', 'Place_NL': 'Cairo', 'data_country': 'egypte', 'data_place': 'cairo'}}}

first_place = next(iter(ddc['egypte']))  # Get the first key in the inner dictionary
ddc['egypte'][first_place]['Country']

'Egypte'

In [321]:
ddc

{'egypte': {'alexandrië': {'Country': 'Egypte',
   'Place': 'Alexandrië',
   'Region': None,
   'Type': 'stad',
   'Lat': 31.1991806,
   'Lon': 29.8951716,
   'Map': 1,
   'Label': 'b',
   'Popular': 1,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Alexandria',
   'Region_EN': None,
   'Continent': 'Afrika',
   'Continent_EN': 'Africa',
   'Country_NL': 'Egypte',
   'Place_NL': 'Alexandrië',
   'data_country': 'egypte',
   'data_place': 'alexandrië'},
  'cairo': {'Country': 'Egypte',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257,
   'Map': 1,
   'Label': 'b',
   'Popular': 1,
   'Superpopular': 0,
   'Country_EN': 'Egypt',
   'Place_EN': 'Cairo',
   'Region_EN': None,
   'Continent': 'Afrika',
   'Continent_EN': 'Africa',
   'Country_NL': 'Egypte',
   'Place_NL': 'Cairo',
   'data_country': 'egypte',
   'data_place': 'cairo'}}}

In [324]:
dict_destinations_data = ddc

dict_countrydata = {}
ct='egypte'
dict_countrydata[ct]={'Country':'','n_popular_ct':0,'places':{}}
# dict_countrydata = {ct:}
n_popular_ct = 0
for p in dict_destinations_data[ct]:
    dict_countrydata[ct]['Country']=dict_destinations_data[ct][p]['Country']
    dict_countrydata[ct]['places'][p]={k:dict_destinations_data[ct][p][k] for k in ['Place','Popular']}
    dict_countrydata[ct]['places'][p]['country']=ct
    dict_countrydata[ct]['places'][p]['place']=p
    n_popular_ct += dict_countrydata[ct]['places'][p]['Popular']
dict_countrydata[ct]['n_popular_ct']=n_popular_ct

dict_countrydata

{'egypte': {'Country': 'Egypte',
  'n_popular_ct': 2,
  'places': {'alexandrië': {'Place': 'Alexandrië',
    'Popular': 1,
    'country': 'egypte',
    'place': 'alexandrië'},
   'cairo': {'Place': 'Cairo',
    'Popular': 1,
    'country': 'egypte',
    'place': 'cairo'}}}}